In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sql
# import pickle
# import dateutil.parser # to parse ISO date
from datetime import datetime, timedelta

# Plotly
import plotly.graph_objects as go

# Web app
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table

database_name = "FX_data/database.db"

conn = sql.connect('FX_data/database.db')
cur = conn.cursor()
df = pd.read_sql_query("SELECT * FROM eda_data_LR ",conn)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'],inplace=True)
conn.close()

# FOR DASH APP
cluster_options = [{'label': i, 'value': i} for i in list(df["Cluster"].unique())]
year_options = pd.DatetimeIndex(df.reset_index()['Date']).year #.year
action_options = [{'label': k, 'value': v} for k,v in {('Sell Signal',0),('Buy Signal',1)}]#[{'label': i, 'value': i} for i in list(df["Prediction"].unique())]
forecast_options = [{'label': k, 'value': v} for k,v in {('Forecasted Wrongly',0),('Forecasted Correctly',1)}]#[{'label': i, 'value': i} for i in list(df["Correct"].unique())]
dataset_options = [{'label':'All Trades','value':'eda_data_LR'}, {'label':'Profitable Subgroups','value':'profitable_subgroups'}]
model_options = [{'label': k, 'value': v} for k,v in {('Logistic Regression',0),('Random Forest',1)}],

#df = pd.read_sql_query("SELECT * FROM profitable_subgroups", conn)
df.head(1)

,Target,Prediction,Correct,% Buy,% Sell,Price,Open,High,Low,Close,Prob Buy,Prob Sell,Balance,Profit,comp_0,comp_1,t-SNE_1,t-SNE_2,Cluster,Confidence %
Date,,,,,,,,,,,,,,,,,,,,
2004-03-01,0,1,0,37.25,62.75,0.8034,0.755824,0.762779,0.758515,0.76933,0.364423,0.635577,9903.846154,-96.153846,-0.019401,0.017532,22.393435,-12.548299,5,62.75


In [67]:
# days_options = pd.DatetimeIndex(df.reset_index()['Date'])

days_options = [{'label': i, 'value': i} for i in pd.DatetimeIndex(df.reset_index()['Date'])]
#str(i)[:-9]
days_options[:5]



# {'points': [{'customdata': '2005-10-10 00:00:00'}]}

[{'label': Timestamp('2004-03-01 00:00:00'),
  'value': Timestamp('2004-03-01 00:00:00')},
 {'label': Timestamp('2004-03-02 00:00:00'),
  'value': Timestamp('2004-03-02 00:00:00')},
 {'label': Timestamp('2004-03-03 00:00:00'),
  'value': Timestamp('2004-03-03 00:00:00')},
 {'label': Timestamp('2004-03-04 00:00:00'),
  'value': Timestamp('2004-03-04 00:00:00')},
 {'label': Timestamp('2004-03-05 00:00:00'),
  'value': Timestamp('2004-03-05 00:00:00')}]

### Layout
- [Learn More link](https://github.com/plotly/dash-sample-apps/blob/master/apps/dash-oil-and-gas/app.py)
- [features Medium example](https://medium.com/@leclercqz.acacio/build-a-dashboard-from-scratch-using-dash-python-framework-7bd0ea986f6)
- [Price charts](https://plot.ly/python/time-series/)
- [Candle sticks](https://plot.ly/python/candlestick-charts/)
##### ISSUES 
- ~~<font color=red> Setting the heigh/width on scatterplot callback interferes with layout </font>~~
    - added __ in left box to increase height 

In [109]:
# DEFINE HTML TEMPLATE
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div(
    [
        dcc.Store(
            id="aggregate_data", 
        ),
        # TITLE BANNER
        html.Div(
            [
                html.Div(
                    [
                        html.H1(
                        " USD/EUR TRADES ",
                        style={"margin-bottom": "0px"},
                        ),
                        html.H3(
                        "Identifying under/over performing trades for improved profitability",
                        #" Model Performance Evaluation ",
                        style={"margin-top": "0px"},
                        #style = {"line-height": "60px"},
                        ),
                    html.Hr(),
                    ],
                    className="ten columns",
                ),
                html.Div(
                    [
                        html.Img(
                            src="https://media-exp1.licdn.com/dms/image/C4D03AQEcxZXx15lL-w/profile-displayphoto-shrink_100_100/0?e=1586995200&v=beta&t=dO0XD0N3rBbGGhCBO2Ds-OZJjAvEPzpwlXZz2ps2LJI",
                            #"https://s3-us-west-1.amazonaws.com/plotly-tutorials/logo/new-branding/dash-logo-by-plotly-stripe.png",
#                             style={
#                                 "height": "60px",
#                                 "width": "auto",
#                                 "margin-bottom": "25px",
#                             },
                        ),
                    #html.Hr(),
                    ],className="two columns",                       
                ),           
                #
            ],
            id="header",
            className = "row flex-display",
            style={"margin-bottom": "25px"},
        ),       
        # USER OPTION PANNEL SEPARATE
        html.Div(       
            [
                html.Div(
                    [
                        html.B(
                            "User Option Panel",
                            #style={"textAlign":"centered"},           # <-------------------  3doesnt work
                            #align="center",
                        ), # name for section (Card)
                        
                        html.Hr(),  # underline for section 
                        #0 Model ( All trades / Profitable Subgroup)   <------------------ NOT MEANT TO WORK
                        html.P(
                            "Select Forecasting Model:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='model_1',
                            options= model_options,
                            value= 0,
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Logistic Regression", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        html.Hr(),                        
                        #1 Dataset ( All trades / Profitable Subgroup)
                        html.P(
                            "Select All Clusters / Profitable Clusters:",
                            className="control_label",
                        ),
                        dcc.Dropdown(
                            id='dataset_1',
                            options= dataset_options,
                            value= 'eda_data_LR',
                            #style={"display": "block","margin-left": "0","width": "40%"},
                            placeholder= "Select a dataset", 
                            searchable=False,
                            clearable=False,
                            className="dcc_control",
                        ),  
                        html.Hr(),
                        #2 Cluster Dropdown 
                        html.P(
                            "Cluster Dropdown:",
                            className="control_panel",
                        ), 
                        dcc.Dropdown(
                            id='cluster_1',
                            options=cluster_options, 
                            multi=True,
                            value= list(range(0,10)), #list(df["Cluster"].unique().sort_values()),
                            #style={"display": "block","margin-left": "0","width": "70%"},
                            className="dcc_control",                      
                        ),  
                        html.Hr(),
                        #3 Trade Actions - Buy/Sell
                        html.P(
                            "Select Trade Type:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='action_1',
                            options=action_options,
                            value= [0,1],                            
                            style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),
                        html.Hr(),
                        # Forecasts - Correct/Wrong
                        html.P(
                            "Filter Correct/False Forecasts:",
                            className="control_panel",
                        ), 
                        dcc.Checklist(
                            id='forecast_1',
                            options=forecast_options,
                            value= [0,1],                            
                            #style={"display": "block","margin-left": "0","width": "30%"},     
                            className="dcc_control",  
                        ),                        
                        html.Hr(),
                        #4 Data range slider - Years
                        html.P("Select Year:",
                            className="control_panel",
                        ),                     
                        dcc.RangeSlider( 
                            id = "date_1",
                            updatemode = "drag", 
                            min = year_options.min(),
                            max = year_options.max(),
                            marks = {i: '{}'.format(i) for i in year_options.unique().tolist()},
                            value= [2005,2006],        
                            className="dcc_control", 
                        ), 
                        html.Hr(),
                        html.Div(
                            id='slider_selection',
#                             style={"display": "block","margin-left": "auto","width": "30%"},
                        ), 
                        
                    ],
                    id= "cross-filter-options",     #'slider_selection',
                    #style={'height': '100%', 'display': 'inline-block'},
                    className="pretty_container four columns",
                    #className="four columns",

                ),
                # SCATTERPLOT SEPARATE
                html.Div(
                    [    
                        html.Div(
                            [
                                html.B("Analyze Trader Subgroups"),
                                html.Hr(),
                                dcc.Graph(
                                    id = "scatter_trader",  
                                    clickData = '2005-10-10', #  #df.index[100], 
                                    style={
#                                            "display": "block",
#                                            "margin-right": "0",
#                                            "width": "70%",
#                                             "height":"700",  <-- -DONT WORK
#                                             "width":"700",
                                          },
                                        ),
                            ],
                            id = "countGraphContainer",
                            #className="pretty_container",
                        ),
                    ],
                    id="rightCol",
                    #style={'width': '70%', 'display': 'inline-block'},
                    className="pretty_container eight columns",
                ),    
            ],
            #className="row",
            className = "row flex-display",
            #style={"margin-bottom": "25px"},
        ),
        
        # PLACE HOLDER FOR 2nd GRAPH
        html.Div(
            [
                # Header
                html.Div(
                    [
                        html.H3(
                        " PRICE CHART ",
                        style = { "margin-bottom": "25px"},
                        #style={"max-width": "100%", "font-size": "1.5rem", "padding": "1px 1px"},
                        ), 
                        
                    ],
                ),
                
                # Graph
                html.Div(
                    [
                        dcc.Graph(
                            id='price_chart',    
                        ), 
                    ],
                ),
                
            ],
            className="row",
        ),
    ],
    style={'backgroundColor':'rgb(240, 240, 240)'},
)   # END 


### Callbacks
- <font color=red> The price chart does not update dynamically, received None data from scatter_trader plot </font>


- [Custom Hover Data](https://community.plot.ly/t/how-to-access-customdata-with-hover-action/26324/6)
- Example 2

In [110]:
@app.callback(
                [
                    Output('scatter_trader', 'figure'),
                    Output('slider_selection', 'children'), 
                    #Output('price_chart', 'figure'),
                  ],
               #Output('table_1', 'data'),  ], # Dynamic data table 
                [
                    Input ('model_1','value'),
                    Input ('dataset_1','value'),
                    Input('cluster_1','value'),
                    Input('action_1','value'), 
                    Input('forecast_1','value'),
                    Input('date_1','value'),           
                ]   
            ) 

def plots (modelD,datasetD, clusterD, actionD, forecastD, rangeD ):  # just my names (According to order in callback)
    
    # Call data from SQL db
    conn = sql.connect(database_name)
    cur = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM {} ".format(datasetD), conn)  #FROM eda_data_LR
    #df["Date_Day"] = str(df['Date']) #pd.to_datetime(df['Date'])
    df["Date_Y"] = pd.DatetimeIndex(df["Date"]).year
       # Just for price chart 
    conn.close()    
    
    #dates = pd.to_datetime(df.Date, format = '%Y%m%d')
    #day_dates = dates.apply(lambda x: x.strftime('%Y-%m-%d'))
    
    dataset = df[(df["Date_Y"] >= rangeD[0]) &  # Slice data according to lower year range 
                   (df["Date_Y"] <= rangeD[1]) & # Slice data according to upper year range
                   (df["Cluster"].isin(clusterD)) & # Slice data according to quarter                       
                   (df["Prediction"].isin(actionD)) &
                   (df["Correct"].isin(forecastD))# Slice data according to sector    
                  ]  
    
    # ============================= 3D Scatterplot =============================== #
    trace1 = go.Scatter3d(x = dataset["t-SNE_1"],  
                          y = dataset["t-SNE_2"],
                          z = dataset["Profit"], 
                          #customdata = dataset["Date_Day"],  # hover data ???
                          mode = "markers+text",
                          #hovertext =  round(dataset[["Profit","Confidence %"]],2).values.tolist(), #dataset[["Profit"]].values.tolist(),  #,"Confidence %"
                          
                          #customdata = dataset["Date"] ,
                          hovertext = dataset["Date"] , #day_dates , #["Date_Day"], #day_dates,
         
                          #hovertext = [ (str(x)) for x in df['Date']],
                          
                          #dataset['Date_Day'], # Just for price chart
                          
                          
                          hoverinfo= "text",
                          marker = dict(size= df["Confidence %"]/4, 
                                        sizemin = 1.5,  # min size
                                        color= dataset["Cluster"],  # THIS CAN BE DIFFERENT
                                        colorscale = "Portland",
                                        colorbar = {"thickness": 10, "len": 0.5, "x": 0.8, "y": 0.6, },
                                        symbol = list(np.where(dataset['Correct'] == 1, 'diamond', 'cross')), #square #-open
                                        showscale = False,
                                       ),  
                         )
    data_1 = [ trace1 ]    
    layout_1 = go.Layout( 
             title = "<b>Scatterplot Trades</b>",
             #clickmode = 'event+select',
             # UX
             height = 850,  #1500,
             #width = 850,  #1500,    <----- Screws with layout 
             paper_bgcolor = "rgb(230, 230, 230)", # background colour "rgb(220, 235, 247)"
             # paper_bgcolor = "rgb(158, 194, 217)", # background colour 
             scene = dict( # UI
                          aspectmode = "cube",  
                          hovermode = "closest",
                          dragmode = "turntable", #  "orbit" | "turntable" | "zoom" | "pan" | False
                          xaxis = dict(title ="t-SNE_1" ),
                          yaxis = dict(title ="t-SNE_2" ),
                          zaxis = dict(title ="Profit"),  
             )
    )   

    fig1 =( 
                    {"data":data_1,
                     "layout":layout_1}
                   )
    # print (dataset["Date"])
    return fig1 , "Date Range {} - {}.".format(str(rangeD[0]),str(rangeD[1])) #, price_chart



    # ============================= Price Chart =============================== #
    
@app.callback(
                    Output('price_chart', 'figure'),

                [

                    Input('scatter_trader','figure'), # Input('scatter_trader','hoverData'),   
                    Input('scatter_trader','clickData'),   
                ],
            ) 

def plots_2 (figure,clickData):  # just my names (According to order in callback)
    
    conn = sql.connect(database_name)
    cur = conn.cursor()
    datasetD =  "eda_data_LR"
    
    conn = sql.connect('FX_data/database.db')
    df = pd.read_sql_query("SELECT * FROM {} ".format(datasetD), conn)
    df['Date'] = pd.to_datetime(df['Date'])
    
    """
    -- GET HOVER DATA --
    dcc.Graph has four attributes --> https://dash.plot.ly/interactive-graphing
    """

    #trade_date_end = pd.to_datetime(app.layout['scatter_trader'].clickData)
    trade_date_end = pd.to_datetime(clickData)
    history_days = 100 # 100 days of previous price data
    trade_date_start = trade_date_end - timedelta(days=history_days) 
    print (trade_date_start, trade_date_end )
    
    """
    https://community.plot.ly/t/get-trace-name-from-hoverdata/7574
    trade_date = app.layout['scatter_trader'].fig1['points'][hoverData]['customdata']
    """

    
    raw_p = pd.read_sql_query("SELECT * FROM raw_prices WHERE Date >= '{}' AND Date <= '{}' ".format(trade_date_start,trade_date_end), conn)  #FROM eda_data_LR
    raw_p['Date']=pd.to_datetime(raw_p['Date'])
    #raw_p.set_index(['Date'],inplace=True)
    conn.close()   
    
    raw_p = raw_p[-100:]
    trace2 = go.Candlestick(x= raw_p['Date'],
                            open=raw_p['Open'].to_list(),
                            high=raw_p['High'].to_list(),
                            low=raw_p['Low'].to_list(),
                            close=raw_p['Price'].to_list(),
                            )
    data_2 = [ trace2 ] 
    
    layout_2 = go.Layout( 
                         title = "<b>Price Chart</b>",
                         xaxis_rangeslider_visible=False,
                         # UX
                         #height = 850,  #1500,
                         #width = 850,  #1500,    <----- Screws with layout 
                         paper_bgcolor = "rgb(230, 230, 230)", # background colour "rgb(220, 235, 247)"
                         # paper_bgcolor = "rgb(158, 194, 217)", # background colour 
#                              scene = dict( # UI
#                                           aspectmode = "cube",  
#                                           hovermode = "closest",
#                                           dragmode = "turntable", #  "orbit" | "turntable" | "zoom" | "pan" | False
#                                           xaxis = dict(title ="t-SNE_1" ),
#                                           yaxis = dict(title ="t-SNE_2" ),
#                                           zaxis = dict(title ="t-SNE_3"),  
#                              )
    )   
    
    fig2 =( {"data":data_2,
             "layout":layout_2}
          )
    
    return fig2
    

if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Feb/2020 12:28:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2020 12:28:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2020 12:28:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2020 12:28:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Feb/2020 12:28:56] "POST /_dash-update-component HTTP/1.1" 200 -


2005-07-02 00:00:00 2005-10-10 00:00:00
2005-07-02 00:00:00 2005-10-10 00:00:00


127.0.0.1 - - [18/Feb/2020 12:28:56] "POST /_dash-update-component HTTP/1.1" 200 -


### Example Candle plot
The dcc.Graph component has four attributes that can change through user-interaction: hoverData, clickData, selectedData, relayoutData

In [97]:
conn = sql.connect(database_name)
cur = conn.cursor()
trade_date_start = str(df.index[1]) # start date not same as raw dataset 
trade_date = str(df.index[100])  #str(df.index[1])[:-4]  #  '2004-03-02 00'
raw_p = pd.read_sql_query("SELECT * FROM raw_prices WHERE Date >= '{}' AND Date <= '{}' ".format(trade_date_start,trade_date), conn)  #FROM eda_data_LR
raw_p['Date']=pd.to_datetime(raw_p['Date'])
#raw_p.set_index(['Date'],inplace=True)
conn.close() 

raw_p.head(1)

,Date,Price,Open,High,Low
0,2004-03-02,0.8184,0.8021,0.8201,0.8017


In [98]:
raw_p

,Date,Price,Open,High,Low
0,2004-03-02,0.8184,0.8021,0.8201,0.8017
1,2004-03-03,0.8190,0.8183,0.8294,0.8178
2,2004-03-04,0.8200,0.8191,0.8240,0.8165
3,2004-03-05,0.8079,0.8195,0.8214,0.8041
4,2004-03-08,0.8063,0.8070,0.8105,0.8052
...,...,...,...,...,...
95,2004-07-13,0.8107,0.8055,0.8130,0.8052
96,2004-07-14,0.8075,0.8104,0.8117,0.8047
97,2004-07-15,0.8092,0.8072,0.8114,0.8067
98,2004-07-16,0.8030,0.8093,0.8113,0.8023


In [99]:
fig = go.Figure(data=[go.Candlestick(x=raw_p['Date'],
                                        open=raw_p['Open'],
                                        high=raw_p['High'],
                                        low=raw_p['Low'],
                                        close=raw_p['Price']
                                            )
                                             ])

fig.update_layout(xaxis_rangeslider_visible=False) # remove range finder

fig.show()